In [1]:
%pip install --upgrade pip
%pip install davos
import davos

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
smuggle pandas as pd # pip: pandas==1.5.1
smuggle numpy as np # pip: numpy==1.23.4
smuggle seaborn as sns # pip: seaborn==0.12.1
smuggle dill as pickle # pip: dill==0.3.6

smuggle h5py # pip: h5py==3.7.0
smuggle pathos # pip: pathos==0.3.0
smuggle quail # pip: quail==0.2.1
smuggle requests # pip: requests==2.28.1
smuggle os
smuggle warnings
smuggle string

from tqdm smuggle tqdm # pip: tqdm==4.64.1
from matplotlib smuggle pyplot as plt #pip: matplotlib==3.6.2
from matplotlib.ticker smuggle MaxNLocator
from pathos.multiprocessing smuggle ProcessingPool as Pool # pip: pathos==0.3.0
from multiprocessing smuggle cpu_count  # pip: multiprocess==0.70.14
from sklearn.decomposition smuggle IncrementalPCA as PCA # pip: scikit-learn==1.1.3
from scipy smuggle stats # pip: scipy==1.10.0

# local functions
from dataloader import datadir, grouping, feature_groupings, descriptions, sort_by_grouping, fetch_data
from analyze import analyze_data, recover_fingerprint_features, organize_by_listgroup, random, adaptive, non_adaptive_exclude_random, \
                    select_conds, select_lists, filter, get_diffs, stack_diffs, pnr_matrix, accuracy2df, adaptive_listnum2cond, \
                    clustering_matrices, average_by_cond, rename_features, fingerprint2temporal, get_boundaries, \
                    recall_accuracy_near_boundaries, results, results_by_list, analyses, listgroups, orders, ttest, merge_results

# Performance on *feature rich* versus *reduced* lists:
  - accuracy
  - temporal clustering
  - non-visual feature-based clustering (category, size, length, first letter)

In [3]:
print('Accuracy for feature rich vs. reduced (all lists):')
ttest(results['accuracy']['feature rich'], results['accuracy']['reduced'])

Accuracy for feature rich vs. reduced (all lists):
t(126) = -0.290, p = 0.772


In [4]:
print('Temporal clustering for feature rich vs. reduced (all lists):')
ttest(results['fingerprint']['feature rich'], results['fingerprint']['reduced'], x_col='temporal', y_col='temporal')

Temporal clustering for feature rich vs. reduced (all lists):
t(126) = 10.624, p = 0.000


In [5]:
features = ['category', 'size', 'wordLength', 'firstLetter']
for i, f in enumerate(features):
    print(f'{f} clustering for feature rich vs. reduced (all lists):')
    ttest(results['fingerprint']['feature rich'], results['fingerprint']['reduced'], x_col=f, y_col=f)

    if i < len(features) - 1:
        print('\n')

category clustering for feature rich vs. reduced (all lists):
t(126) = 10.077, p = 0.000


size clustering for feature rich vs. reduced (all lists):
t(126) = 11.829, p = 0.000


wordLength clustering for feature rich vs. reduced (all lists):
t(126) = 10.639, p = 0.000


firstLetter clustering for feature rich vs. reduced (all lists):
t(126) = 7.775, p = 0.000


# Performance on {*feature rich*, *reduced*} lists versus *reduced ({early, late})* lists
We'll compare early and late lists separately (e.g., early to early, late to late, early to late, etc.)

Metrics:
  - accuracy
  - temporal clustering
  - non-visual feature-based clustering (category, size, length, first letter)

## Early vs. late (all conditions and metrics)

### Accuracy

In [6]:
print('Accuracy for feature rich (early lists) vs. feature rich (late lists):')
ttest(results['accuracy']['feature rich'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Late', independent_sample=False)

Accuracy for feature rich (early lists) vs. feature rich (late lists):
t(66) = 4.553, p = 0.000


In [7]:
print('Accuracy for reduced (early lists) vs. reduced (late lists):')
ttest(results['accuracy']['reduced'], results['accuracy']['reduced'], x_lists='Early', y_lists='Late', independent_sample=False)

Accuracy for reduced (early lists) vs. reduced (late lists):
t(60) = 2.434, p = 0.018


In [8]:
print('Accuracy for reduced (early) (early lists) vs. reduced (early) (late lists):')
ttest(results['accuracy']['reduced (early)'], results['accuracy']['reduced (early)'], x_lists='Early', y_lists='Late', independent_sample=False)

Accuracy for reduced (early) (early lists) vs. reduced (early) (late lists):
t(41) = 1.499, p = 0.141


In [9]:
print('Accuracy for reduced (late) (early lists) vs. reduced (late) (late lists):')
ttest(results['accuracy']['reduced (late)'], results['accuracy']['reduced (late)'], x_lists='Early', y_lists='Late', independent_sample=False)

Accuracy for reduced (late) (early lists) vs. reduced (late) (late lists):
t(40) = 1.462, p = 0.152


### Temporal clustering

In [10]:
print('Temporal clustering for feature rich (early lists) vs. feature rich (late lists):')
ttest(results['fingerprint']['feature rich'], results['fingerprint']['feature rich'], x_col='temporal', y_col='temporal', x_lists='Early', y_lists='Late', independent_sample=False)

Temporal clustering for feature rich (early lists) vs. feature rich (late lists):
t(66) = 2.318, p = 0.024


In [11]:
print('Temporal clustering for reduced (early lists) vs. reduced (late lists):')
ttest(results['fingerprint']['reduced'], results['fingerprint']['reduced'], x_col='temporal', y_col='temporal', x_lists='Early', y_lists='Late', independent_sample=False)

Temporal clustering for reduced (early lists) vs. reduced (late lists):
t(60) = 0.929, p = 0.357


In [12]:
print('Temporal clustering for reduced (early) (early lists) vs. reduced (early) (late lists):')
ttest(results['fingerprint']['reduced (early)'], results['fingerprint']['reduced (early)'], x_col='temporal', y_col='temporal', x_lists='Early', y_lists='Late', independent_sample=False)

Temporal clustering for reduced (early) (early lists) vs. reduced (early) (late lists):
t(41) = 0.998, p = 0.324


In [13]:
print('Temporal clustering for reduced (late) (early lists) vs. reduced (late) (late lists):')
ttest(results['fingerprint']['reduced (late)'], results['fingerprint']['reduced (late)'], x_col='temporal', y_col='temporal', x_lists='Early', y_lists='Late', independent_sample=False)

Temporal clustering for reduced (late) (early lists) vs. reduced (late) (late lists):
t(40) = 1.099, p = 0.278


### Non-visual feature based clustering

In [14]:
for i, f in enumerate(features):
    print(f'{f} clustering for feature rich (early lists) vs. feature rich (late lists):')
    ttest(results['fingerprint']['feature rich'], results['fingerprint']['feature rich'], x_col=f, y_col=f, x_lists='Early', y_lists='Late', independent_sample=False)

    if i < len(features) - 1:
        print('\n')

category clustering for feature rich (early lists) vs. feature rich (late lists):
t(66) = 3.805, p = 0.000


size clustering for feature rich (early lists) vs. feature rich (late lists):
t(66) = 2.190, p = 0.032


wordLength clustering for feature rich (early lists) vs. feature rich (late lists):
t(66) = 0.161, p = 0.872


firstLetter clustering for feature rich (early lists) vs. feature rich (late lists):
t(66) = 0.410, p = 0.683


In [15]:
for i, f in enumerate(features):
    print(f'{f} clustering for reduced (early lists) vs. reduced (late lists):')
    ttest(results['fingerprint']['reduced'], results['fingerprint']['reduced'], x_col=f, y_col=f, x_lists='Early', y_lists='Late', independent_sample=False)

    if i < len(features) - 1:
        print('\n')

category clustering for reduced (early lists) vs. reduced (late lists):
t(60) = 2.856, p = 0.006


size clustering for reduced (early lists) vs. reduced (late lists):
t(60) = 2.947, p = 0.005


wordLength clustering for reduced (early lists) vs. reduced (late lists):
t(60) = 3.528, p = 0.001


firstLetter clustering for reduced (early lists) vs. reduced (late lists):
t(60) = 2.275, p = 0.026


In [16]:
for i, f in enumerate(features):
    print(f'{f} clustering for reduced (early) (early lists) vs. reduced (early) (late lists):')
    ttest(results['fingerprint']['reduced (early)'], results['fingerprint']['reduced (early)'], x_col=f, y_col=f, x_lists='Early', y_lists='Late', independent_sample=False)

    if i < len(features) - 1:
        print('\n')

category clustering for reduced (early) (early lists) vs. reduced (early) (late lists):
t(41) = 0.753, p = 0.456


size clustering for reduced (early) (early lists) vs. reduced (early) (late lists):
t(41) = 0.721, p = 0.475


wordLength clustering for reduced (early) (early lists) vs. reduced (early) (late lists):
t(41) = 0.493, p = 0.625


firstLetter clustering for reduced (early) (early lists) vs. reduced (early) (late lists):
t(41) = 0.780, p = 0.440


In [17]:
for i, f in enumerate(features):
    print(f'{f} clustering for reduced (late) (early lists) vs. reduced (late) (late lists):')
    ttest(results['fingerprint']['reduced (late)'], results['fingerprint']['reduced (late)'], x_col=f, y_col=f, x_lists='Early', y_lists='Late', independent_sample=False)

    if i < len(features) - 1:
        print('\n')

category clustering for reduced (late) (early lists) vs. reduced (late) (late lists):
t(40) = -0.086, p = 0.932


size clustering for reduced (late) (early lists) vs. reduced (late) (late lists):
t(40) = 0.746, p = 0.460


wordLength clustering for reduced (late) (early lists) vs. reduced (late) (late lists):
t(40) = 1.476, p = 0.148


firstLetter clustering for reduced (late) (early lists) vs. reduced (late) (late lists):
t(40) = 0.966, p = 0.340


## Feature rich vs. reduced ({early, late})

### Accuracy

In [18]:
print('Accuracy for feature rich vs. reduced (early) (all lists):')
ttest(results['accuracy']['feature rich'], results['accuracy']['reduced (early)'])

Accuracy for feature rich vs. reduced (early) (all lists):
t(107) = -2.230, p = 0.028


In [19]:
print('Accuracy for feature rich vs. reduced (late) (all lists):')
ttest(results['accuracy']['feature rich'], results['accuracy']['reduced (late)'])

Accuracy for feature rich vs. reduced (late) (all lists):
t(106) = -0.638, p = 0.525


### Temporal clustering

In [20]:
print('Temporal clustering for feature rich vs. reduced (early) (all lists):')
ttest(results['fingerprint']['feature rich'], results['fingerprint']['reduced (early)'], x_col='temporal', y_col='temporal')

Temporal clustering for feature rich vs. reduced (early) (all lists):
t(107) = -1.434, p = 0.154


In [21]:
print('Temporal clustering for feature rich vs. reduced (late) (all lists):')
ttest(results['fingerprint']['feature rich'], results['fingerprint']['reduced (late)'], x_col='temporal', y_col='temporal')

Temporal clustering for feature rich vs. reduced (late) (all lists):
t(106) = -0.536, p = 0.593


### Non-visual feature based clustering

In [22]:
features = ['category', 'size', 'wordLength', 'firstLetter']
for i, f in enumerate(features):
    print(f'{f} clustering for feature rich vs. reduced (early) (all lists):')
    ttest(results['fingerprint']['feature rich'], results['fingerprint']['reduced (early)'], x_col=f, y_col=f)

    if i < len(features) - 1:
        print('\n')

category clustering for feature rich vs. reduced (early) (all lists):
t(107) = -0.216, p = 0.829


size clustering for feature rich vs. reduced (early) (all lists):
t(107) = -0.481, p = 0.632


wordLength clustering for feature rich vs. reduced (early) (all lists):
t(107) = -1.002, p = 0.319


firstLetter clustering for feature rich vs. reduced (early) (all lists):
t(107) = -0.534, p = 0.595


In [23]:
features = ['category', 'size', 'wordLength', 'firstLetter']
for i, f in enumerate(features):
    print(f'{f} clustering for feature rich vs. reduced (late) (all lists):')
    ttest(results['fingerprint']['feature rich'], results['fingerprint']['reduced (late)'], x_col=f, y_col=f)

    if i < len(features) - 1:
        print('\n')

category clustering for feature rich vs. reduced (late) (all lists):
t(106) = -1.316, p = 0.191


size clustering for feature rich vs. reduced (late) (all lists):
t(106) = -1.304, p = 0.195


wordLength clustering for feature rich vs. reduced (late) (all lists):
t(106) = -1.359, p = 0.177


firstLetter clustering for feature rich vs. reduced (late) (all lists):
t(106) = 0.850, p = 0.397


## Reduced vs. reduced ({early, late})

### Accuracy

In [24]:
print('Accuracy for reduced vs. reduced (early) (all lists):')
ttest(results['accuracy']['reduced'], results['accuracy']['reduced (early)'])

Accuracy for reduced vs. reduced (early) (all lists):
t(101) = -2.045, p = 0.043


In [25]:
print('Accuracy for reduced vs. reduced (late) (all lists):')
ttest(results['accuracy']['reduced'], results['accuracy']['reduced (late)'])

Accuracy for reduced vs. reduced (late) (all lists):
t(100) = -0.407, p = 0.685


### Temporal clustering

In [26]:
print('Temporal clustering for reduced vs. reduced (early) (all lists):')
ttest(results['fingerprint']['reduced'], results['fingerprint']['reduced (early)'], x_col='temporal', y_col='temporal')

Temporal clustering for reduced vs. reduced (early) (all lists):
t(101) = -10.789, p = 0.000


In [27]:
print('Temporal clustering for reduced vs. reduced (late) (all lists):')
ttest(results['fingerprint']['reduced'], results['fingerprint']['reduced (late)'], x_col='temporal', y_col='temporal')

Temporal clustering for reduced vs. reduced (late) (all lists):
t(100) = -9.780, p = 0.000


### Non-visual feature based clustering

In [28]:
features = ['category', 'size', 'wordLength', 'firstLetter']
for i, f in enumerate(features):
    print(f'{f} clustering for reduced vs. reduced (early) (all lists):')
    ttest(results['fingerprint']['reduced'], results['fingerprint']['reduced (early)'], x_col=f, y_col=f)

    if i < len(features) - 1:
        print('\n')

category clustering for reduced vs. reduced (early) (all lists):
t(101) = -9.770, p = 0.000


size clustering for reduced vs. reduced (early) (all lists):
t(101) = -12.149, p = 0.000


wordLength clustering for reduced vs. reduced (early) (all lists):
t(101) = -10.722, p = 0.000


firstLetter clustering for reduced vs. reduced (early) (all lists):
t(101) = -7.331, p = 0.000


In [29]:
features = ['category', 'size', 'wordLength', 'firstLetter']
for i, f in enumerate(features):
    print(f'{f} clustering for reduced vs. reduced (late) (all lists):')
    ttest(results['fingerprint']['reduced'], results['fingerprint']['reduced (late)'], x_col=f, y_col=f)

    if i < len(features) - 1:
        print('\n')

category clustering for reduced vs. reduced (late) (all lists):
t(100) = -10.232, p = 0.000


size clustering for reduced vs. reduced (late) (all lists):
t(100) = -12.107, p = 0.000


wordLength clustering for reduced vs. reduced (late) (all lists):
t(100) = -9.691, p = 0.000


firstLetter clustering for reduced vs. reduced (late) (all lists):
t(100) = -5.443, p = 0.000


# Order manipulation analyses

- When lists are sorted by a given feature, how is memory performance affected (relative to feature rich -- early lists only)?
- Do some order manipulations matter more than others?  E.g. compare semantic vs. lexicographic vs. visual -- early lists only

### Accuracy

In [30]:
print('Accuracy for category vs. feature rich (early lists):')
ttest(results['accuracy']['category'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

print('\nAccuracy for size vs. feature rich (early lists):')
ttest(results['accuracy']['size'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

print('\nAccuracy for length vs. feature rich (early lists):')
ttest(results['accuracy']['length'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

print('\nAccuracy for first letter vs. feature rich (early lists):')
ttest(results['accuracy']['first letter'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

print('\nAccuracy for color vs. feature rich (early lists):')
ttest(results['accuracy']['color'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

print('\nAccuracy for location vs. feature rich (early lists):')
ttest(results['accuracy']['location'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

Accuracy for category vs. feature rich (early lists):
t(95) = 3.034, p = 0.003

Accuracy for size vs. feature rich (early lists):
t(95) = -1.013, p = 0.314

Accuracy for length vs. feature rich (early lists):
t(95) = -0.550, p = 0.584

Accuracy for first letter vs. feature rich (early lists):
t(95) = -0.690, p = 0.492

Accuracy for color vs. feature rich (early lists):
t(96) = 1.850, p = 0.067

Accuracy for location vs. feature rich (early lists):
t(95) = 0.043, p = 0.966


### Temporal clustering

In [31]:
print('Temporal clustering for category vs. feature rich (early lists):')
ttest(results['fingerprint']['category'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for size vs. feature rich (early lists):')
ttest(results['fingerprint']['size'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for length vs. feature rich (early lists):')
ttest(results['fingerprint']['length'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for first letter vs. feature rich (early lists):')
ttest(results['fingerprint']['first letter'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for color vs. feature rich (early lists):')
ttest(results['fingerprint']['color'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for location vs. feature rich (early lists):')
ttest(results['fingerprint']['location'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

Temporal clustering for category vs. feature rich (early lists):
t(95) = 8.508, p = 0.000

Temporal clustering for size vs. feature rich (early lists):
t(95) = 2.429, p = 0.017

Temporal clustering for length vs. feature rich (early lists):
t(95) = -1.666, p = 0.099

Temporal clustering for first letter vs. feature rich (early lists):
t(95) = 2.587, p = 0.011

Temporal clustering for color vs. feature rich (early lists):
t(96) = -1.064, p = 0.290

Temporal clustering for location vs. feature rich (early lists):
t(95) = 1.682, p = 0.096


### Feature-based clustering

In [32]:
conds = ['category', 'size', 'length', 'first letter', 'color', 'location']
features = ['category', 'size', 'wordLength', 'firstLetter', 'color', 'location']

for i, c in enumerate(conds):
    for j, f in enumerate(features):
        print(f'{f} clustering for {c} vs. feature rich (early lists):')
        ttest(results['fingerprint'][c], results['fingerprint']['feature rich'], x_col=f, y_col=f, x_lists='Early', y_lists='Early')

        if (j < len(features) - 1) or (i < len(conds) - 1):
            print('\n')
    
    if i < len(conds) - 1:
        print('--- \n')

category clustering for category vs. feature rich (early lists):
t(95) = 4.305, p = 0.000


size clustering for category vs. feature rich (early lists):
t(95) = 3.777, p = 0.000


wordLength clustering for category vs. feature rich (early lists):
t(95) = 0.457, p = 0.649


firstLetter clustering for category vs. feature rich (early lists):
t(95) = 0.105, p = 0.917


color clustering for category vs. feature rich (early lists):
t(95) = -0.126, p = 0.900


location clustering for category vs. feature rich (early lists):
t(95) = -0.099, p = 0.922


--- 

category clustering for size vs. feature rich (early lists):
t(95) = 0.187, p = 0.852


size clustering for size vs. feature rich (early lists):
t(95) = 1.895, p = 0.061


wordLength clustering for size vs. feature rich (early lists):
t(95) = 0.384, p = 0.702


firstLetter clustering for size vs. feature rich (early lists):
t(95) = -0.998, p = 0.321


color clustering for size vs. feature rich (early lists):
t(95) = -0.806, p = 0.422


lo

## Combine "categories" of features:
  - semantic = category + size
  - lexicographic = length + first letter
  - visual = color + location

In [42]:
groups = {k: v for k, v in feature_groupings.items() if k in ['semantic', 'lexicographic', 'visual']}

merged_results = {}
for k, v in results.items():
    if k not in ['fingerprint', 'accuracy']:
        continue
    merged_results[k] = merge_results(v, groups)

## Compare semantic vs. feature rich

### Accuracy

In [46]:
print('Accuracy for semantic vs. feature rich (early lists):')
ttest(merged_results['accuracy']['semantic'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

print('\nAccuracy for lexicographic vs. feature rich (early lists):')
ttest(merged_results['accuracy']['lexicographic'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

print('\nAccuracy for visual vs. feature rich (early lists):')
ttest(merged_results['accuracy']['visual'], results['accuracy']['feature rich'], x_lists='Early', y_lists='Early')

Accuracy for semantic vs. feature rich (early lists):
t(125) = 1.197, p = 0.233

Accuracy for lexicographic vs. feature rich (early lists):
t(125) = -0.776, p = 0.439

Accuracy for visual vs. feature rich (early lists):
t(126) = 1.256, p = 0.212


### Temporal clustering

In [48]:
print('Temporal clustering for semantic vs. feature rich (early lists):')
ttest(merged_results['fingerprint']['semantic'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for lexicographic vs. feature rich (early lists):')
ttest(merged_results['fingerprint']['lexicographic'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for visual vs. feature rich (early lists):')
ttest(merged_results['fingerprint']['visual'], results['fingerprint']['feature rich'], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

Temporal clustering for semantic vs. feature rich (early lists):
t(125) = 6.183, p = 0.000

Temporal clustering for lexicographic vs. feature rich (early lists):
t(125) = 0.549, p = 0.584

Temporal clustering for visual vs. feature rich (early lists):
t(126) = 0.405, p = 0.686


### Feature based clustering

In [50]:
conds = ['semantic', 'lexicographic', 'visual']
features = ['category', 'size', 'wordLength', 'firstLetter', 'color', 'location']

for i, c in enumerate(conds):
    for j, f in enumerate(features):
        print(f'{f} clustering for {c} vs. feature rich (early lists):')
        ttest(merged_results['fingerprint'][c], results['fingerprint']['feature rich'], x_col=f, y_col=f, x_lists='Early', y_lists='Early')

        if (j < len(features) - 1) or (i < len(conds) - 1):
            print('\n')
    
    if i < len(conds) - 1:
        print('--- \n')

category clustering for semantic vs. feature rich (early lists):
t(125) = 2.524, p = 0.013


size clustering for semantic vs. feature rich (early lists):
t(125) = 3.510, p = 0.001


wordLength clustering for semantic vs. feature rich (early lists):
t(125) = 0.539, p = 0.591


firstLetter clustering for semantic vs. feature rich (early lists):
t(125) = -0.587, p = 0.558


color clustering for semantic vs. feature rich (early lists):
t(125) = -0.579, p = 0.564


location clustering for semantic vs. feature rich (early lists):
t(125) = -0.346, p = 0.730


--- 

category clustering for lexicographic vs. feature rich (early lists):
t(125) = -1.078, p = 0.283


size clustering for lexicographic vs. feature rich (early lists):
t(125) = -0.310, p = 0.757


wordLength clustering for lexicographic vs. feature rich (early lists):
t(125) = 3.426, p = 0.001


firstLetter clustering for lexicographic vs. feature rich (early lists):
t(125) = 3.236, p = 0.002


color clustering for lexicographic vs. f

## Compare each pair of (category of) order manipulation conditions (semantic, lexicographic, and visual)

### Accuracy

In [60]:
for i, c1 in enumerate(conds):
    for c2 in conds[i+1:]:
        print(f'Accuracy for {c1} vs. {c2} (early lists):')
        ttest(merged_results['accuracy'][c1], merged_results['accuracy'][c2], x_lists='Early', y_lists='Early')

        print(f'\nAccuracy for {c1} vs. {c2} (late lists):')
        ttest(merged_results['accuracy'][c1], merged_results['accuracy'][c2], x_lists='Late', y_lists='Late')

        print('\n\n')

Accuracy for semantic vs. lexicographic (early lists):
t(118) = 1.936, p = 0.055

Accuracy for semantic vs. lexicographic (late lists):
t(118) = -0.388, p = 0.699



Accuracy for semantic vs. visual (early lists):
t(119) = 0.113, p = 0.910

Accuracy for semantic vs. visual (late lists):
t(119) = -0.833, p = 0.407



Accuracy for lexicographic vs. visual (early lists):
t(119) = -2.145, p = 0.034

Accuracy for lexicographic vs. visual (late lists):
t(119) = -0.352, p = 0.726





### Temporal clustering

In [61]:
for i, c1 in enumerate(conds):
    for c2 in conds[i+1:]:
        print(f'Temporal clustering for {c1} vs. {c2} (early lists):')
        ttest(merged_results['fingerprint'][c1], merged_results['fingerprint'][c2], x_lists='Early', y_lists='Early', x_col='temporal', y_col='temporal')

        print(f'\nTemporal clustering for {c1} vs. {c2} (late lists):')
        ttest(merged_results['fingerprint'][c1], merged_results['fingerprint'][c2], x_lists='Late', y_lists='Late', x_col='temporal', y_col='temporal')

        print('\n')

Temporal clustering for semantic vs. lexicographic (early lists):
t(118) = 5.572, p = 0.000

Temporal clustering for semantic vs. lexicographic (late lists):
t(118) = -0.693, p = 0.490


Temporal clustering for semantic vs. visual (early lists):
t(119) = 6.215, p = 0.000

Temporal clustering for semantic vs. visual (late lists):
t(119) = -0.308, p = 0.759


Temporal clustering for lexicographic vs. visual (early lists):
t(119) = 0.189, p = 0.850

Temporal clustering for lexicographic vs. visual (late lists):
t(119) = 0.505, p = 0.615




### Feature-based clustering

In [63]:
for i, c1 in enumerate(conds):
    for c2 in conds[i+1:]:
        for f in features:
            print(f'{f} clustering for {c1} vs. {c2} (early lists):')
            ttest(merged_results['fingerprint'][c1], merged_results['fingerprint'][c2], x_col=f, y_col=f, x_lists='Early', y_lists='Early')

            print(f'\n{f} clustering for {c1} vs. {c2} (late lists):')
            ttest(merged_results['fingerprint'][c1], merged_results['fingerprint'][c2], x_col=f, y_col=f, x_lists='Late', y_lists='Late')

            print('\n')

category clustering for semantic vs. lexicographic (early lists):
t(118) = 3.492, p = 0.001

category clustering for semantic vs. lexicographic (late lists):
t(118) = -0.734, p = 0.464


size clustering for semantic vs. lexicographic (early lists):
t(118) = 3.972, p = 0.000

size clustering for semantic vs. lexicographic (late lists):
t(118) = -1.628, p = 0.106


wordLength clustering for semantic vs. lexicographic (early lists):
t(118) = -3.112, p = 0.002

wordLength clustering for semantic vs. lexicographic (late lists):
t(118) = 1.199, p = 0.233


firstLetter clustering for semantic vs. lexicographic (early lists):
t(118) = -3.686, p = 0.000

firstLetter clustering for semantic vs. lexicographic (late lists):
t(118) = -1.025, p = 0.307


color clustering for semantic vs. lexicographic (early lists):
t(118) = -0.388, p = 0.699

color clustering for semantic vs. lexicographic (late lists):
t(118) = -0.606, p = 0.546


location clustering for semantic vs. lexicographic (early lists):
t

# When *early* lists are sorted by a given feature, how is memory performance on *late* lists affected (relative to *feature rich* late lists)

### Accuracy

In [57]:
print('Accuracy for semantic vs. feature rich (late lists):')
ttest(merged_results['accuracy']['semantic'], results['accuracy']['feature rich'], x_lists='Late', y_lists='Late')

print('\nAccuracy for lexicographic vs. feature rich (late lists):')
ttest(merged_results['accuracy']['lexicographic'], results['accuracy']['feature rich'], x_lists='Late', y_lists='Late')

print('\nAccuracy for visual vs. feature rich (late lists):')
ttest(merged_results['accuracy']['visual'], results['accuracy']['feature rich'], x_lists='Late', y_lists='Late')

Accuracy for semantic vs. feature rich (late lists):
t(125) = 0.487, p = 0.627

Accuracy for lexicographic vs. feature rich (late lists):
t(125) = 0.878, p = 0.382

Accuracy for visual vs. feature rich (late lists):
t(126) = 1.437, p = 0.153


### Temporal clustering

In [58]:
print('Temporal clustering for semantic vs. feature rich (late lists):')
ttest(merged_results['fingerprint']['semantic'], results['fingerprint']['feature rich'], x_lists='Late', y_lists='Late', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for lexicographic vs. feature rich (late lists):')
ttest(merged_results['fingerprint']['lexicographic'], results['fingerprint']['feature rich'], x_lists='Late', y_lists='Late', x_col='temporal', y_col='temporal')

print('\nTemporal clustering for visual vs. feature rich (late lists):')
ttest(merged_results['fingerprint']['visual'], results['fingerprint']['feature rich'], x_lists='Late', y_lists='Late', x_col='temporal', y_col='temporal')

Temporal clustering for semantic vs. feature rich (late lists):
t(125) = 0.146, p = 0.884

Temporal clustering for lexicographic vs. feature rich (late lists):
t(125) = 0.923, p = 0.358

Temporal clustering for visual vs. feature rich (late lists):
t(126) = 0.525, p = 0.601


## Feature-based clustering

In [66]:
conds = ['semantic', 'lexicographic', 'visual']
features = ['category', 'size', 'wordLength', 'firstLetter', 'color', 'location']

for i, c in enumerate(conds):
    for j, f in enumerate(features):
        print(f'{f} clustering for {c} vs. feature rich (late lists):')
        ttest(merged_results['fingerprint'][c], results['fingerprint']['feature rich'], x_col=f, y_col=f, x_lists='Late', y_lists='Late')

        if (j < len(features) - 1) or (i < len(conds) - 1):
            print('\n')
    
    if i < len(conds) - 1:
        print('--- \n')

category clustering for semantic vs. feature rich (late lists):
t(125) = -0.066, p = 0.947


size clustering for semantic vs. feature rich (late lists):
t(125) = -0.832, p = 0.407


wordLength clustering for semantic vs. feature rich (late lists):
t(125) = -0.009, p = 0.993


firstLetter clustering for semantic vs. feature rich (late lists):
t(125) = -0.899, p = 0.370


color clustering for semantic vs. feature rich (late lists):
t(125) = -0.570, p = 0.570


location clustering for semantic vs. feature rich (late lists):
t(125) = -0.537, p = 0.592


--- 

category clustering for lexicographic vs. feature rich (late lists):
t(125) = 0.673, p = 0.502


size clustering for lexicographic vs. feature rich (late lists):
t(125) = 0.808, p = 0.421


wordLength clustering for lexicographic vs. feature rich (late lists):
t(125) = -1.234, p = 0.220


firstLetter clustering for lexicographic vs. feature rich (late lists):
t(125) = 0.223, p = 0.824


color clustering for lexicographic vs. feature r


- when lists are sorted by a given feature, how is memory performance affected (relative to feature rich -- early lists only)?
  - do some order manipulations matter more than others?  e.g. compare semantic vs. lexicographic vs. visual -- early lists only


- are order effects "sticky"?
  - when *early* lists are sorted by a given feature, how is memory performance on *late* lists affected (relative to feature rich -- late lists only)?
  - do some order manipulations matter more than others?  e.g. compare semantic vs. lexicographic vs. visual -- late lists only

  - compare behavior on early vs. late lists for order manipulation condition
    - does feature clustering on early lists correlate with recall on early (or late) lists?
    - does feature clustering on late lists correlate with recall on early (or late) lists?
    - (ditto, but replace "recall" with "temporal clustering")


- for feature-rich lists, do order effects matter?
  - recall + dynamics + organization on order-manipulation conditions vs. feature rich
  - fingerprint trajectories: how much do fingerprints change over time, are they sensitive to order manipulations?

- are fingerprints maleable?  how does match between fingerprint + presentation order affect recall performance (adaptive conditions)?